### Network

In this section the `rnn.py` file is explained in more details. The `RNNModel` class receives as parameter the following:

- `input_size`: the same size as the one-hot encoded array.
- `output_size`: as our expected output is the same size as the input, the output will be the same size than the one-hot encoded array.
- `hidden_dim`: the size of features in the hidden state.
- `n_layers`: the number of RNN layers to be used.

The line 15 defines the RNN layer. `batch_first` is necessary to specify the way in which data are represented in the input. In this example, the first dimension represents the batch size, the second the sentence and the third each character in the setence represented as an one-hot array.

After the RNN layer, the data will passes through a fully connected layer to present the output, as shown on line 19. The fully connected layer receives the output from the recurrent layer. Therefore, the input size will be the same as the number of features in the hidden state (`hidden_dim`). The output size of the fully connected layer will be the same size as the output sentence one-hot encoded.

Bellow is the whole code of the recurrent neural network.

In [ ]:
import torch
import torch.nn as nn


class RNNModel(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(RNNModel, self).__init__()

        # Defining some parameters.
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        # region Defining the layers.
        # RNN layer.
        self.rnn = nn.RNN(
            input_size, hidden_dim, n_layers, batch_first=True, nonlinearity="relu"
        )
        # Fully connected layer.
        self.fc = nn.Linear(hidden_dim, output_size)
        # endregion

    def forward(self, x):
        batch_size = x.size(axis=0)

        # Initializing hidden state for first input using method defined below.
        hidden = self.init_hidden(batch_size)  # (1, 3, 12)

        # Passing in the input and hidden state into the model and obtaining outputs.
        out, hidden = self.rnn(x, hidden)  # (3, 14, 12), (1, 3, 12)

        # Reshaping the outputs such that it can be fit into the fully connected layer.
        out = out.contiguous().view(-1, self.hidden_dim)  # (42, 12)
        out = self.fc(out)  # (42, 17)

        return out, hidden

    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we will use in the forward pass.
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).cuda()
        return hidden

The forward method is responsible for passing the input through the network layers. The first step is to initialize the hidden state for the recurrent layer. The shape of the hidden state will be the number of recurrent layers, the size of the batch and the number of hidden features.

The image below shows in more details the size of the inputs and outputs shapes through the network layers.

![shapes](../../documentation/tensor-sizes-through-network.png)

As you can see, the network is simple. Therefore, there will be problems with overfitting. The goal is just to understand the simplest concepts about recurrent neural networks.